<a href="https://colab.research.google.com/github/KevinCY-Kim/AI_Study/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%98_Basic_%EA%B3%A0%EA%B0%9D_%EC%A7%80%EC%9B%90_%EB%93%B1%EA%B8%89_%EB%B6%84%EB%A5%98_py%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split

# =======================
# 1. 데이터 로드
# =======================
train = pd.read_csv("/content/train.csv", sep=",")
test = pd.read_csv("/content/test.csv", sep=",")

X = train.drop(columns=["ID", "support_needs"])
y = train["support_needs"]

X_test = test.drop(columns=["ID"])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [2]:
print("Head of train:")
display(train.head())

print("\nHead of test:")
display(test.head())

print("\nHead of X:")
display(X.head())

print("\nHead of y:")
display(y.head())

print("\nHead of X_train:")
display(X_train.head())

print("\nHead of X_valid:")
display(X_valid.head())

print("\nHead of y_train:")
display(y_train.head())

print("\nHead of y_valid:")
display(y_valid.head())

Head of train:


,ID,age,gender,tenure,frequent,payment_interval,subscription_type,contract_length,after_interaction,support_needs
0,TRAIN_00000,54.0,F,47.0,22.0,8.0,member,90,25.0,0
1,TRAIN_00001,30.0,M,16.0,15.0,5.0,vip,360,23.0,0
2,TRAIN_00002,29.0,M,8.0,30.0,21.0,plus,30,21.0,0
3,TRAIN_00003,38.0,F,38.0,23.0,10.0,vip,90,6.0,0
4,TRAIN_00004,25.0,F,52.0,3.0,17.0,member,30,1.0,2



Head of test:


,ID,age,gender,tenure,frequent,payment_interval,subscription_type,contract_length,after_interaction
0,TEST_00000,18.0,M,40.0,6.0,15.0,member,30,18.0
1,TEST_00001,40.0,M,41.0,23.0,0.0,member,90,16.0
2,TEST_00002,59.0,F,30.0,1.0,21.0,member,360,25.0
3,TEST_00003,38.0,M,2.0,10.0,0.0,member,30,18.0
4,TEST_00004,30.0,M,28.0,21.0,20.0,member,360,28.0



Head of X:


...

,age,gender,tenure,frequent,payment_interval,subscription_type,contract_length,after_interaction
10527,56.0,M,56.0,16.0,6.0,member,30,10.0
15103,62.0,F,10.0,20.0,16.0,member,360,7.0
2626,41.0,M,57.0,21.0,7.0,member,90,23.0
28108,48.0,F,37.0,8.0,9.0,plus,90,5.0
885,46.0,M,47.0,30.0,1.0,vip,360,1.0



Head of y_train:


,support_needs
11378,0
18706,0
23515,2
11239,2
15659,0



Head of y_valid:


,support_needs
10527,1
15103,1
2626,0
28108,1
885,2


In [3]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

# =======================================================
# 1. 클러스터링에 사용할 피처 선택 및 데이터 통합
# =======================================================

# 범주형 변수는 이 단계에서 제외, 클러스터링 피처 선택 (수치형 변수만)
# 모든 데이터셋을 합쳐서 (Train + Validation + Test) 클러스터링에 사용할 피처를 준비합니다.
# Object 타입(문자열/범주형) 및 Target 변수(support_needs)는 제외하고 수치형 변수만 사용합니다.
# ID는 이미 drop되었으므로 고려하지 않습니다.
# 비지도 학습이므로, 별도의 Target도 필요하지 않음.

# =======================================================
# 1. 클러스터링에 사용할 피처 선택 및 데이터 통합 (인덱스 초기화)
# =======================================================
CLUSTER_FEATURES = ['age', 'tenure', 'frequent', 'payment_interval', 'contract_length', 'after_interaction']

# 클러스터링 수행을 위해 세트들을 통합하고 인덱스를 초기화합니다.
# 인덱스 초기화를 통해 중복 레이블 문제를 근본적으로 해결합니다.

# 원본 인덱스 정보를 보존하기 위한 준비
train_idx = X_train.index
valid_idx = X_valid.index
test_idx = X_test.index

# 클러스터링에 사용할 데이터만 모으고 인덱스를 0부터 다시 시작하도록 초기화합니다.
X_combined_data = pd.concat([X_train[CLUSTER_FEATURES],
                             X_valid[CLUSTER_FEATURES],
                             X_test[CLUSTER_FEATURES]],
                            axis=0).reset_index(drop=True)

# =======================================================
# 2. 표준화 (Scaling) 수행 (수정된 부분: 인덱스 초기화된 데이터 사용)
# =======================================================
scaler = StandardScaler()

# 결측치 처리 (X_combined_data를 기준으로 처리)
X_combined_filled = X_combined_data.fillna(X_combined_data.mean())

# X_train 부분만 슬라이싱하여 Scaling 학습 (Data Leakage 방지)
train_size = len(X_train)
valid_size = len(X_valid)

X_train_part = X_combined_filled.iloc[:train_size]
X_valid_part = X_combined_filled.iloc[train_size : train_size + valid_size]
X_test_part = X_combined_filled.iloc[train_size + valid_size :]


# (1) X_train_part에 fit_transform 후, 전체에 transform 적용
X_train_scaled = scaler.fit_transform(X_train_part)
X_combined_scaled = scaler.transform(X_combined_filled)


# =======================================================
# 3. K-Means 모델 학습 및 클러스터 라벨링
# =======================================================
K = 5
kmeans = KMeans(n_clusters=K,
                random_state=42,
                n_init='auto',
                tol=1e-04)

# 학습 데이터셋(X_train_scaled)을 기준으로 클러스터 모델 학습
kmeans.fit(X_train_scaled)

# 통합 데이터셋에 클러스터 예측 라벨을 부여
cluster_labels_combined = kmeans.predict(X_combined_scaled)

# =======================================================
# 4. 결과: 클러스터 ID 파생변수 추가 (수정된 부분: 인덱스 초기화된 데이터프레임 이용)
# =======================================================

# 클러스터 라벨을 인덱스가 초기화된 데이터프레임으로 생성
df_combined_cluster = pd.DataFrame(cluster_labels_combined, columns=['Cluster_ID'])

# 각 분할된 데이터셋 크기에 맞게 클러스터 ID를 슬라이싱
cluster_train = df_combined_cluster.iloc[:train_size]['Cluster_ID']
cluster_valid = df_combined_cluster.iloc[train_size : train_size + valid_size]['Cluster_ID']
cluster_test = df_combined_cluster.iloc[train_size + valid_size :]['Cluster_ID']

# 원본 데이터프레임(X_train, X_valid, X_test)에 새로운 Cluster_ID를 할당
# 이 때, 할당되는 Cluster_ID의 인덱스를 원본 데이터프레임의 인덱스에 맞춥니다.
X_train['Cluster_ID'] = cluster_train.values
X_valid['Cluster_ID'] = cluster_valid.values
X_test['Cluster_ID'] = cluster_test.values

X_train['Cluster_ID'] = X_train['Cluster_ID'].astype('category')
X_valid['Cluster_ID'] = X_valid['Cluster_ID'].astype('category')
X_test['Cluster_ID'] = X_test['Cluster_ID'].astype('category')

print(f"✅ K-Means 클러스터링이 완료되었으며, K={K}인 'Cluster_ID' 파생변수가 중복 인덱스 오류 없이 추가되었습니다.")
print("\n[X_train의 Cluster_ID 분포]")
print(X_train['Cluster_ID'].value_counts().sort_index())

✅ K-Means 클러스터링이 완료되었으며, K=5인 'Cluster_ID' 파생변수가 중복 인덱스 오류 없이 추가되었습니다.

[X_train의 Cluster_ID 분포]
Cluster_ID
0    4394
1    4827
2    4458
3    6307
4    4700
Name: count, dtype: int64


In [4]:
%pip install pycaret
# Pycaret지원되는 파이썬 런타임 유형으로 변경해야함(2025.07 - 3.11.14)

In [5]:
import pandas as pd
from pycaret.classification import * # pycaret 라이브러리에서 classification 모듈을 가져옵니다.

# =======================================================
# 1. 학습용 데이터셋 통합
# =======================================================
# X_train과 y_train을 하나의 데이터프레임으로 결합합니다.
# PyCaret의 setup 함수는 전체 데이터프레임을 요구합니다.
train_data_for_pycaret = X_train.copy()
train_data_for_pycaret['support_needs'] = y_train

print(f"PyCaret 학습 데이터 크기: {train_data_for_pycaret.shape}")

# =======================================================
# 2. PyCaret 환경 설정 (setup)
# =======================================================
# setup() 함수는 PyCaret 환경을 초기화하고 데이터 파이프라인을 구축합니다.
# Target Type: support_needs가 여러 범주를 가지므로 Multi-class classification으로 자동 인식됩니다.
# session_id: 재현성을 위해 고정합니다.
# categorical_features: 명시적으로 범주형 변수를 지정하여 정확한 인코딩을 유도합니다.
#                       (Cluster_ID는 이전 단계에서 category 타입으로 지정했으나, 명시적으로 지정해줍니다.)

# 범주형 변수 목록 (Cluster_ID 포함)
CATEGORICAL_FEATURES = ['gender', 'subscription_type', 'Cluster_ID']

clf_setup = setup(data = train_data_for_pycaret,
                  target = 'support_needs',
                  session_id = 42,
                  categorical_features = CATEGORICAL_FEATURES,
                  # normalize=True, # 스케일링은 K-Means 단계에서 이미 수행했으므로 선택적으로 사용합니다.
                  fold = 5, # 교차 검증 폴드 수를 5로 설정 (기본값은 10)
                  n_jobs = -1, # 가능한 모든 코어 사용
                  verbose = True) # 설정 과정을 출력합니다.

# =======================================================
# 3. 모델 비교 및 상위 3개 모델 선정 (compare_models)
# =======================================================
# compare_models() 함수는 PyCaret의 모든 모델을 교차 검증으로 학습하고 비교합니다.
# n_select=3: 상위 3개 모델을 리스트로 반환하도록 설정합니다.
# sort='Accuracy' 또는 'F1': 모델 순위 기준이 됩니다. (이 다중 분류 문제에서는 'F1' 또는 'Accuracy'가 일반적)

top3_models = compare_models(n_select=3,
                             sort='F1')

print("\n\n✅ PyCaret AutoML을 통해 선정된 상위 3개 모델 목록 (F1 Score 기준):")
print(top3_models)

PyCaret 학습 데이터 크기: (24686, 10)


,Description,Value
0,Session id,42
1,Target,support_needs
2,Target type,Multiclass
3,Original data shape,"(24686, 10)"
4,Transformed data shape,"(24686, 16)"
5,Transformed train set shape,"(17280, 16)"
6,Transformed test set shape,"(7406, 16)"
7,Numeric features,6
8,Categorical features,3
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.5115,0.6655,0.5115,0.4792,0.4837,0.2116,0.2177,7.4060
lightgbm,Light Gradient Boosting Machine,0.5278,0.6785,0.5278,0.4882,0.4812,0.2384,0.2518,0.7340
xgboost,Extreme Gradient Boosting,0.5017,0.6600,0.5017,0.4743,0.4799,0.1978,0.2020,0.8760
rf,Random Forest Classifier,0.4997,0.6554,0.4997,0.4681,0.4730,0.1933,0.1987,1.7400
gbc,Gradient Boosting Classifier,0.5337,0.0000,0.5337,0.4827,0.4673,0.2405,0.2612,3.6280
et,Extra Trees Classifier,0.4876,0.6393,0.4876,0.4546,0.4614,0.1676,0.1726,1.5400
knn,K Neighbors Classifier,0.4646,0.6065,0.4646,0.4417,0.4405,0.1217,0.1263,0.5740
ada,Ada Boost Classifier,0.5042,0.0000,0.5042,0.4435,0.4334,0.1571,0.1809,0.4560
dt,Decision Tree Classifier,0.4212,0.5518,0.4212,0.4241,0.4226,0.1010,0.1010,0.1980
lda,Linear Discriminant Analysis,0.4969,0.0000,0.4969,0.4376,0.4216,0.1376,0.1629,0.1240


Processing:   0%|          | 0/71 [00:00<?, ?it/s]



✅ PyCaret AutoML을 통해 선정된 상위 3개 모델 목록 (F1 Score 기준):
[<catboost.core.CatBoostClassifier object at 0x798fb0c8c210>, LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0), XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cpu', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
            

In [6]:
# 전방(Base) 모델 학습 및 선정

# =======================================================
# 1. 상위 3개 모델 생성 (Base Estimators)
# =======================================================

# 1위: Light Gradient Boosting Machine (lightgbm)
lgbm = create_model('lightgbm', fold=5)
# 2위: Extreme Gradient Boosting (xgboost)
xgb = create_model('xgboost', fold=5)
# 3위: Random Forest Classifier (rf)
rf = create_model('rf', fold=5)

# 상위 3개 모델을 리스트로 묶습니다. (이들이 Stacker의 '입력' 역할을 합니다)
base_models = [lgbm, xgb, rf]

print("✅ 상위 3개 기반 모델 학습 완료.")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5333,0.6826,0.5333,0.4918,0.4864,0.2451,0.2587
1,0.5243,0.6773,0.5243,0.4856,0.4769,0.2350,0.2487
2,0.5327,0.6799,0.5327,0.4996,0.4871,0.2449,0.2587
3,0.5249,0.6800,0.5249,0.4762,0.4783,0.2347,0.2470
4,0.5237,0.6725,0.5237,0.4879,0.4772,0.2324,0.2458
Mean,0.5278,0.6785,0.5278,0.4882,0.4812,0.2384,0.2518
Std,0.0043,0.0034,0.0043,0.0076,0.0046,0.0054,0.0057


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5095,0.6656,0.5095,0.4840,0.4894,0.2098,0.2139
1,0.5000,0.6588,0.5000,0.4731,0.4803,0.1996,0.2028
2,0.5110,0.6610,0.5110,0.4818,0.4870,0.2114,0.2165
3,0.4942,0.6622,0.4942,0.4654,0.4713,0.1841,0.1884
4,0.4939,0.6525,0.4939,0.4671,0.4717,0.1843,0.1886
Mean,0.5017,0.6600,0.5017,0.4743,0.4799,0.1978,0.2020
Std,0.0073,0.0044,0.0073,0.0075,0.0075,0.0118,0.0120


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5072,0.6612,0.5072,0.4749,0.4808,0.2035,0.2091
1,0.4986,0.6552,0.4986,0.4682,0.4728,0.1943,0.1994
2,0.5029,0.6588,0.5029,0.4690,0.4744,0.1968,0.2027
3,0.4977,0.6527,0.4977,0.4666,0.4710,0.1905,0.1959
4,0.4922,0.6490,0.4922,0.4618,0.4659,0.1814,0.1866
Mean,0.4997,0.6554,0.4997,0.4681,0.4730,0.1933,0.1987
Std,0.0051,0.0043,0.0051,0.0042,0.0048,0.0073,0.0075


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

✅ 상위 3개 기반 모델 학습 완료.


In [7]:
%pip install catboost

In [8]:
# CatBoost 메타 모델 설정 및 Stacking 진행

# =======================================================
# 2. CatBoost 모델 생성 (Meta Model / Final Estimator)
# =======================================================
# CatBoost Classifier를 메타 모델로 생성합니다.
# 'catboost'는 PyCaret에서 'catboost'라는 ID로 사용됩니다.
# (이 때, CatBoost는 PyCaret의 setup 과정에서 자동으로 설치되거나 수동 설치가 필요할 수 있습니다.)

catb = create_model('catboost', fold=5, verbose=False) # 상세 출력은 끄고 생성


# =======================================================
# 3. Stacking 앙상블 모델 생성
# =======================================================

# stack_models 함수를 사용하여 Stacking 앙상블을 수행합니다.
# estimator_list: 상위 3개 모델 (입력 모델)
# meta_model: CatBoost (최종 결합 모델)

# 참고: 이 과정은 시간이 다소 소요될 수 있습니다.
stacker_catb = stack_models(estimator_list = base_models,
                            meta_model = catb,
                            fold = 5)

print("\n\n✅ CatBoost를 메타 모델로 사용한 Stacking 앙상블 모델 학습 완료.")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5284,0.6802,0.5284,0.4976,0.4926,0.2414,0.2514
1,0.5171,0.6705,0.5171,0.4816,0.4802,0.2265,0.2359
2,0.5168,0.6672,0.5168,0.4699,0.4749,0.2225,0.2326
3,0.5127,0.6664,0.5127,0.4700,0.4713,0.2160,0.2262
4,0.5029,0.6639,0.5029,0.4652,0.4662,0.2023,0.2107
Mean,0.5156,0.6696,0.5156,0.4769,0.4770,0.2218,0.2314
Std,0.0082,0.0057,0.0082,0.0117,0.0090,0.0128,0.0133


Processing:   0%|          | 0/6 [00:00<?, ?it/s]



✅ CatBoost를 메타 모델로 사용한 Stacking 앙상블 모델 학습 완료.


In [9]:
# (데이터 로드, Cluster_ID 추가, PyCaret setup이 모두 완료된 후 실행)

from pycaret.classification import create_model, stack_models

# 1. 상위 3개 기반 모델 생성 (Base Estimators)
lgbm = create_model('lightgbm', fold=5)
xgb = create_model('xgboost', fold=5)
rf = create_model('rf', fold=5)
base_models = [lgbm, xgb, rf]

# 2. CatBoost 모델 생성 (Meta Model / Final Estimator)
catb = create_model('catboost', fold=5, verbose=False)

# 3. Stacking 앙상블 모델 생성
stacker_catb = stack_models(estimator_list = base_models,
                            meta_model = catb,
                            fold = 5)

print("\n\n✅ Stacking 앙상블 모델 학습 성공!")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5333,0.6826,0.5333,0.4918,0.4864,0.2451,0.2587
1,0.5243,0.6773,0.5243,0.4856,0.4769,0.2350,0.2487
2,0.5327,0.6799,0.5327,0.4996,0.4871,0.2449,0.2587
3,0.5249,0.6800,0.5249,0.4762,0.4783,0.2347,0.2470
4,0.5237,0.6725,0.5237,0.4879,0.4772,0.2324,0.2458
Mean,0.5278,0.6785,0.5278,0.4882,0.4812,0.2384,0.2518
Std,0.0043,0.0034,0.0043,0.0076,0.0046,0.0054,0.0057


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5095,0.6656,0.5095,0.4840,0.4894,0.2098,0.2139
1,0.5000,0.6588,0.5000,0.4731,0.4803,0.1996,0.2028
2,0.5110,0.6610,0.5110,0.4818,0.4870,0.2114,0.2165
3,0.4942,0.6622,0.4942,0.4654,0.4713,0.1841,0.1884
4,0.4939,0.6525,0.4939,0.4671,0.4717,0.1843,0.1886
Mean,0.5017,0.6600,0.5017,0.4743,0.4799,0.1978,0.2020
Std,0.0073,0.0044,0.0073,0.0075,0.0075,0.0118,0.0120


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5072,0.6612,0.5072,0.4749,0.4808,0.2035,0.2091
1,0.4986,0.6552,0.4986,0.4682,0.4728,0.1943,0.1994
2,0.5029,0.6588,0.5029,0.4690,0.4744,0.1968,0.2027
3,0.4977,0.6527,0.4977,0.4666,0.4710,0.1905,0.1959
4,0.4922,0.6490,0.4922,0.4618,0.4659,0.1814,0.1866
Mean,0.4997,0.6554,0.4997,0.4681,0.4730,0.1933,0.1987
Std,0.0051,0.0043,0.0051,0.0042,0.0048,0.0073,0.0075


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5284,0.6802,0.5284,0.4976,0.4926,0.2414,0.2514
1,0.5171,0.6705,0.5171,0.4816,0.4802,0.2265,0.2359
2,0.5168,0.6672,0.5168,0.4699,0.4749,0.2225,0.2326
3,0.5127,0.6664,0.5127,0.4700,0.4713,0.2160,0.2262
4,0.5029,0.6639,0.5029,0.4652,0.4662,0.2023,0.2107
Mean,0.5156,0.6696,0.5156,0.4769,0.4770,0.2218,0.2314
Std,0.0082,0.0057,0.0082,0.0117,0.0090,0.0128,0.0133


Processing:   0%|          | 0/6 [00:00<?, ?it/s]



✅ Stacking 앙상블 모델 학습 성공!
